# Výpočet kolokací vlastní funkcí podle Diceova koeficientu

In [1]:
import nltk
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


## Motivační příklad

Slova "holy" a "am" se v textu *Monty Python and the Holy Grail* (`text6`) obě objevují asi 20×, přičemž "holy" je skoro vždy v kombinaci s (bezprostředně následujícím) "grail", a "am" je skoro vždy v kombinaci s (bezprostředně předcházejícím) "I".

In [2]:
text6.concordance("holy")

Displaying 25 of 26 matches:
[ angels sing ] Arthur , this is the Holy Grail . Look well , Arthur , for it 
rpose , Arthur ... the quest for the Holy Grail . [ boom ] [ singing stops ] L
 he can join us in our quest for the Holy Grail . FRENCH GUARD : Well , I ' ll
gy was required if the quest for the Holy Grail were to be brought to a succes
GALAHAD : You are the keepers of the Holy Grail ? ZOOT : The what ? GALAHAD : 
CELOT : No , we ' ve got to find the Holy Grail . Come on ! GALAHAD : Oh , let
uld be the sign that leads us to the Holy Grail ! Brave , brave Concorde , you
ung people in the joyful bond of the holy wedlock . Unfortunately , one of the
' ve not given up your quest for the Holy Grail ? MINSTREL : [ singing ] He is
 TIM : I do . [ zoosh ] You seek the Holy Grail ! ARTHUR : That is our quest .
, we ' re -- we ' re looking for the Holy Grail . Our quest is to find the Hol
oly Grail . Our quest is to find the Holy Grail . KNIGHTS : Yeah . Yes . It is
 TIM : Yes , I can help

In [3]:
text6.concordance("am")

Displaying 23 of 23 matches:
# 1 : Pull the other one ! ARTHUR : I am , ... and this is my trusty servant P
ght ? ARTHUR : Please ! SOLDIER # 1 : Am I right ? ARTHUR : I ' m not interest
 like an inferior ! ARTHUR : Well , I am king ! DENNIS : Oh king , eh , very n
RTHUR : How do you do , good lady . I am Arthur , King of the Britons . Who ' 
 all are . We are all Britons , and I am your king . WOMAN : I didn ' t know w
HUR : Please , please good people . I am in haste . Who lives in that castle ?
oes he think he is ? Heh . ARTHUR : I am your king ! WOMAN : Well , I didn ' t
bur . [ singing stops ] That is why I am your king ! DENNIS : Listen , strange
f many men , Sir Knight . [ pause ] I am Arthur , King of the Britons . [ paus
e in the ways of science ? ARTHUR : I am Arthur , King of the Britons . BEDEVE
IN : Ah . W -- well , actually I -- I am a Knight of the Round Table . ALL HEA
Knight of the Round Table ? ROBIN : I am . LEFT HEAD : In that case I shall ha
 Well , I -- I -- uh --

Který z těchto **bigramů** (dvojic sousedících slov) intuitivně tvoří silnější **kolokaci**?

Z pohledu gramatiky by se mohlo zdát, že je to "I am" -- cítíme, že jde o spojení slov takřka učebnicové (skutečně ve smyslu, že na ně narazíme v učebnicích). Ovšem sílu kolokace bychom měli posuzovat spíše z hlediska lexikonu, tj. do jaké míry vytváří sledovaná jednotka ucelené a ustálené víceslovné **pojmenování** či **frázi**. Pro tento pohled jsou důležitá dvě kritéria -- zkoumané spojení slov by mělo být:

1. **frekventované** (o jisté míře ustálenosti, tedy lexikalizace, lze uvažovat pouze u jednotek, které se v textech vyskytují opakovaně)
2. **výlučné** (slova v rámci spojení by se měla spolu vyskytovat nápadně častěji než ve společnosti jiných slov)

Když na "I am" a "holy grail" pohlédneme touto optikou, zjistíme, že obě spojení jsou podobně frekventovaná (kolem 20 výskytů), ale spojení "I am" je mnohem méně výlučné, neboť slovo "I" se v textu vyskytuje celkem 260× (tj. ve většině výskytů je v okolí jiných slov než "am"):

In [4]:
text6.concordance("I")

Displaying 25 of 260 matches:
Halt ! Who goes there ? ARTHUR : It is I , Arthur , son of Uther Pendragon , fr
ER # 1 : Pull the other one ! ARTHUR : I am , ... and this is my trusty servant
 will join me in my court at Camelot . I must speak with your lord and master .
t ? ARTHUR : Please ! SOLDIER # 1 : Am I right ? ARTHUR : I ' m not interested 
 ! SOLDIER # 1 : Am I right ? ARTHUR : I ' m not interested ! SOLDIER # 2 : It 
s my point . SOLDIER # 2 : Oh , yeah , I agree with that . ARTHUR : Will you as
T - MASTER : Ninepence . DEAD PERSON : I ' m not dead ! CART - MASTER : What ? 
ere ' s your ninepence . DEAD PERSON : I ' m not dead ! CART - MASTER : ' Ere .
! CUSTOMER : Yes he is . DEAD PERSON : I ' m not ! CART - MASTER : He isn ' t ?
soon . He ' s very ill . DEAD PERSON : I ' m getting better ! CUSTOMER : No you
ead in a moment . CART - MASTER : Oh , I can ' t take him like that . It ' s ag
 s against regulations . DEAD PERSON : I don ' t want to go on the cart ! CUSTO
n ' t be s

Naopak "grail" je v textu 39×, z čehož plyne, že souvýskyty se slovem "holy" tvoří mnohem signifikantnější proporci jeho výskytů. "holy grail" je tedy v textu `text6` silnější kolokací než "I am".

In [5]:
text6.concordance("grail")

Displaying 25 of 39 matches:
els sing ] Arthur , this is the Holy Grail . Look well , Arthur , for it is you
 it is your sacred task to seek this grail . That is your purpose , Arthur ... 
 , Arthur ... the quest for the Holy Grail . [ boom ] [ singing stops ] LAUNCEL
an join us in our quest for the Holy Grail . FRENCH GUARD : Well , I ' ll ask h
ARTHUR : If you will not show us the Grail , we shall take your castle by force
s required if the quest for the Holy Grail were to be brought to a successful c
should separate , and search for the Grail individually . [ clop clop clop ] No
AD : You are the keepers of the Holy Grail ? ZOOT : The what ? GALAHAD : The Gr
il ? ZOOT : The what ? GALAHAD : The Grail . It is here . ZOOT : Oh , but you a
ease ! In God ' s name , show me the Grail ! ZOOT : Oh , you have suffered much
rment me no longer . I have seen the Grail ! PIGLET : There ' s no grail here .
en the Grail ! PIGLET : There ' s no grail here . GALAHAD : I have seen it ! I 
are you goi

## Implementace

Abychom mohli kvantifikovat, *jak přesně* silnější, musíme použít nějakou **asociační** neboli **kolokační míru**. Měrou, která dobře zohledňuje obě výše uvedená kritéria (frekventovanost i výlučnost), je tzv. **Diceův koeficient** (resp. k plnému zohlednění kritéria frekventovanosti je dobré málo frekventované bigramy za kandidáty na kolokace vůbec nepovažovat). Jeho definici můžeme vyčíst z literatury nebo např. [z wiki ÚČNK](http://wiki.korpus.cz/doku.php/pojmy:asociacni_miry?redirect=1#dice_a_logdice), níže je uvedená jedna z možných implementací v Pythonu.

**POZN.:** Není to implementace nejefektivnější (např. kvůli tomu, že spočítáme frekvenční distribuci všech unigramů i bigramů v textu, ačkoli potřebujeme znát jen frekvence dvou cílových slov a jejich bigramu), ale konceptuálně nejlépe odpovídá popisu uvedenému na wiki, takže jako první pokus o převod Diceova koeficientu do Pythonu by měla mít tu výhodu, že je tam relativně malá pravděpodobnost, že člověk někde udělá chybu. Doporučuju porovnat si definici Diceova koeficientu na wiki s touto implementací, abyste se sami přesvědčili, že se jedná skutečně o tentýž výpočet.

In [6]:
def dice_score(word1, word2, text):
    word1, word2 = word1.lower(), word2.lower()
    bigram_fd = nltk.FreqDist(nltk.bigrams(w.lower() for w in text))
    unigram_fd = nltk.FreqDist(w.lower() for w in text)
    return 2 * bigram_fd[(word1, word2)] / (unigram_fd[word1] + unigram_fd[word2])

# POZN.: funkce nltk.bigrams() si jako argument vezme seznam a vrací postupně všechny
# páry sousedících položek:
list(nltk.bigrams([1, 2, 3, 4, 5]))

[(1, 2), (2, 3), (3, 4), (4, 5)]

Diceův koeficient nabývá hodnot mezi 0 a 1; **čím vyšší** číslo, **tím výlučnější** daná kombinace je. Můžeme si tedy exaktně ověřit náš intuitivní dojem, že "holy grail" je silnější kolokací než "I am":

In [7]:
dice_score("holy", "grail", text6)

0.5846153846153846

In [8]:
dice_score("I", "am", text6)

0.14840989399293286

Když máme hotovou prvotní spolehlivou implementaci Diceova koeficientu, můžeme zvážit, zda se nám vyplatí implementovat i hůře srozumitelnou, zato efektivnější verzi. Jaké má původní interpretace nevýhody?

1. je pomalá, protože korpus procházíme zbytečně dvakrát -- poprvé když vytváříme frekvenční distribuci bigramů, podruhé pro frekvenční distribuci unigramů; přitom by mělo být možné získat kýžené informace jediným průchodem korpusu
2. zabírá zbytečně moc paměti, protože vytváříme kompletní frekvenční distribuce všech bigramů a unigramů v korpusu, ačkoli nás zajímají jen dva unigramy a jeden bigram

Na základě těchto poznatků můžeme tedy zkusit implementovat novou, úspornější verzi:

In [9]:
def dice_score2(word1, word2, text):
    word1, word2 = word1.lower(), word2.lower()
    bigram_f = 0
    word1_f = 0
    word2_f = 0
    for b1, b2 in nltk.bigrams(text):
        b1, b2 = b1.lower(), b2.lower()
        if (word1, word2) == (b1, b2):
            bigram_f += 1
        if b1 == word1:
            word1_f += 1
        elif b1 == word2:
            word2_f += 1
    if b2 == word1:
        word1_f += 1
    elif b2 == word2:
        word2_f += 1
    return 2 * bigram_f / (word1_f + word2_f)

Ta už je složitější a hůře srozumitelná -- nepoužívá intuitivně pojmenované funkce jako `FreqDist`, ale generický for-cyklus, je tedy potřeba do detailu přečíst celý kód, aby člověk získal představu, co se v něm děje. Je tedy také vyšší pravděpodobnost, že v kódu uděláme nějakou chybu (už jen kvůli tomu, že je delší -- každé ťuknutí do klávesnice představuje příležitost pro chybu). Proto je dobře, že máme k dispozici dříve definovanou a "spolehlivější" funkci `dice_score`: můžeme porovnat výsledky obou verzí a ujistit se tak, že jsou stejné a že v té nové jsme neudělali chybu:

In [10]:
dice_score("holy", "grail", text6), dice_score2("holy", "grail", text6)

(0.5846153846153846, 0.5846153846153846)

In [11]:
dice_score("I", "am", text6), dice_score2("I", "am", text6)

(0.14840989399293286, 0.14840989399293286)

Výhodou `dice_score2` by mělo být, že zabere méně paměti a bude rychlejší. Rychlost můžeme otestovat pomocí speciálně anotace `%%timeit`, která danou buňku spustí vícekrát a zobrazí, jak dlouho v průměru trvalo operace v buňce vykonat:

In [12]:
%%timeit
dice_score("holy", "grail", text6)

10 loops, best of 3: 35.5 ms per loop


In [13]:
%%timeit
dice_score2("holy", "grail", text6)

100 loops, best of 3: 16.5 ms per loop


Vidíme, že `dice_score2` je skutečně o něco rychlejší, byť ne až zas tak dramaticky (na takto krátkých textech to výrazně nepocítíme). Jak jsme na tom s pamětí? Využití paměti operacemi uvnitř buňky můžeme změřit pomocí anotace `%%memit`, která ale není zabudovanou součástí Jupyteru, musíme ji nejprve nahrát:

In [11]:
%load_ext memory_profiler

In [12]:
%%memit
dice_score("holy", "grail", text6)

peak memory: 150.93 MiB, increment: 0.80 MiB


In [13]:
%%memit
dice_score2("holy", "grail", text6)

peak memory: 150.93 MiB, increment: 0.00 MiB


Vidíme, že první verze navýší spotřebu paměti asi o 0.8 MiB (viz položka *increment*), kdežto druhá v podstatě vůbec. Vzhledem k celkové spotřebě paměti (*peak memory*) jde ale znovu o zanedbatelný rozdíl.

Poučení tedy zní (slovy známého počítačového vědce Donalda Knutha): **Premature optimization is the root of all evil.** Neboli optimalizujte kód (= snažte se upravit implementaci, aby byla účinnější) teprve ve chvíli, kdy vám její pomalost začne vadit jako uživatelům. Neformálně by se tato poučka dala také přeložit "váš čas jakožto programátora je drahocennější než čas počítače" -- takže klidně použijte implementaci, která bude sice pomalejší / paměťožravější, ale bude spolehlivě dobře, než abyste se trápili s vychytáváním chyb (*debugováním*) v implementaci, která bude nepatrně rychlejší / úspornější.

# Na závěr

Může též nastat situace, kdy předpočítat kompletní frekvenční distribuce unigramů i bigramů bude tím úspornějším řešením -- a to v případě, že nechceme spočítat asociační míru pro konkrétní pár slov, ale vytáhnout z textu všechny bigramy s hodnotou koeficientu Dice vyšší než nějaký práh:

In [17]:
from operator import itemgetter

# parametr threshold udává práh koeficientu Dice, přes nějž se musí bigram
# přehoupnout, abychom jej považovali za kolokaci (zde jako defaultní hodnota
# celkem arbitrárně zvoleno 0.5); min_freq je požadovaná minimální frekvence
# bigramu, abychom ho vůbec považovali za kandidáta na kolokace (zde znovu
# celkem arbitrárně 3)

def dice_collocations(text, threshold=.5, min_freq=3):
    bigram_fd = nltk.FreqDist((w1.lower(), w2.lower()) for w1, w2 in nltk.bigrams(text))
    unigram_fd = nltk.FreqDist(w.lower() for w in text)
    collocations = []
    for bigram, freq in bigram_fd.items():
        if freq >= min_freq:
            word1, word2 = bigram
            dice = 2 * bigram_fd[bigram] / (unigram_fd[word1] + unigram_fd[word2])
            if dice >= threshold:
                collocations.append((bigram, dice, freq))
    # na závěr ještě kolokace seřadíme sestupně (reverse=True) podle koeficientu
    # Dice a frekvence (protože seřazujeme seznam n-tic podle hodnoty druhého a
    # třetíhu členu n-tice, musíme použít argument key=itemgetter(1, 2), jehož
    # fungování si osvětlíme na jedné z příštích hodin)
    collocations.sort(key=itemgetter(1, 2), reverse=True)
    return collocations

In [18]:
collocations = dice_collocations(text6)
collocations[:5]

[(('iesu', 'domine'), 1.0, 10),
 (('pie', 'iesu'), 1.0, 10),
 (('dramatic', 'chord'), 1.0, 7),
 (('oral', 'sex'), 1.0, 3),
 (('eis', 'requiem'), 0.9333333333333333, 7)]

# Post scriptum

Napadá vás ještě, jak funkci pro výpočet koeficientu Dice vylepšit? Mohli bychom např. zkusit uživateli umožnit, aby si mohl vybrat, že kolokáty mohou být i v širším kontextu (ne nutně jen těsně vedle sebe), případně dokonce bez ohledu na pořadí. Jistě vás napadnou další možná rozšíření.